# Pandas - ExcelWriter
> Version 0.13 of Pandas added support for new Excel writer engines in addtion to the two engines supported in pervious versions: Xlwt and Openpyxl. The first of the new writer engines to be added is XlsxWriter.

- XlsxWriter is a Python module for writing files in the Excel 2007+ XLSX file format
- XlsxWriter is a fully featured Excel writer that supports options such as autofilters, conditional formatting and charts.

It can be used to write text, numbers, and formulas to multiple worksheets and it supports features such as formatting, images, charts, page setup, autofilters, conditional formatting and many others.

XlsxWriter has some advantages and disadvantages over the alternative Python modules for writing Excel files.

**Advantages:**

- It supports more Excel features than any of the alternative modules.
- It has a high degree of fidelity with files produced by Excel. In most cases the files produced are 100% equivalent to files produced by Excel.
- It has extensive documentation, example files and tests.
- It is fast and can be configured to use very little memory even for very large output files.  


**Disadvantages:**

- It cannot read or modify existing Excel XLSX files.

Link to XlsxWriter : https://xlsxwriter.readthedocs.io/introduction.html


**Example :**

In [1]:
#import pandas with alias pd
import pandas as pd

In [14]:
#create a pandas dataframe from the data 
df_covid_cases = pd.read_csv("owid-covid-data.csv")
# df_covid_cases['total_cases'].fillna(0)
# df_covid_cases['total_cases'] =df_covid_cases['total_cases'].fillna(0).astype(int)
# df_covid_cases
# df_covid_cases.groupby('location').total_cases.sum().astype(int)
df_total_cases = df_covid_cases.groupby('location').total_cases.sum().astype(int)
df_total_cases

location
Afghanistan          3285231
Albania               457133
Algeria              2527633
Andorra               130685
Angola                 87047
                     ...    
Western Sahara         47064
World             1590404440
Yemen                 133576
Zambia                455303
Zimbabwe              232805
Name: total_cases, Length: 212, dtype: int32

In [15]:
# Create a Pandas Excel Writer using XlsxWriter as the engine
new_excel_file = "total_cases.xlsx"
new_sheet_name = "location-wise-totalcases"


writer = pd.ExcelWriter(new_excel_file, engine="xlsxwriter")
df_total_cases.to_excel(writer,sheet_name = new_sheet_name)

# Close the Pandas Excel writer and output the excel file.
writer.save()

## Pandas - Charts

In order to add a chart to the worksheet, access to the underlying XlsxWriter Workbook and Worksheet objects are required.



In [16]:
writer = pd.ExcelWriter(new_excel_file, engine="xlsxwriter")
df_total_cases.to_excel(writer,sheet_name = new_sheet_name)

# Get the workbook
workbook = writer.book 

#get the sheet name
worksheet = writer.sheets[new_sheet_name]

# Create a chart object.
chart = workbook.add_chart({'type':'column'})

# configure the series of the chart from the dataframe data
chart.add_series({
    'values':"='"+new_sheet_name+"'!$B$2:$B$213",
    'categories':"='"+new_sheet_name+"'!$A$2:$A$213",
    'gap':2
})

#insert the chart into the worksheet
worksheet.insert_chart('D2', chart)

writer.save()

![Value_category](https://xlsxwriter.readthedocs.io/_images/chart_axes01.png)

## Axis Labels 

Adding Axis labels to the chart
```
chart.set_x_axis()  
chart.set_y_axis()
```

In [17]:
writer = pd.ExcelWriter(new_excel_file, engine="xlsxwriter")
df_total_cases.to_excel(writer,sheet_name = new_sheet_name)

# Get the workbook
workbook = writer.book 
#get the sheet name
worksheet = writer.sheets[new_sheet_name]

# Create a chart object.
chart = workbook.add_chart({'type':'column'})

# configure the series of the chart from the dataframe data
chart.add_series({
    'values':"='"+new_sheet_name+"'!$B$2:$B$213",
    'categories':"='"+new_sheet_name+"'!$A$2:$A$213",
    'gap':2
})

## add axis labels
chart.set_x_axis({
        'name':'Countries',
        'num_font':{'rotation':45}
})
chart.set_y_axis({'name':'Total Cases','major_gridlines':{'visible':False}})

#insert the chart into the worksheet
worksheet.insert_chart('D2', chart)

writer.save()

## Line Chart

```chart = workbook.add_chart({'type':'line'})```  

![Line chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_line.png)

In [31]:
#get nepal data
# df_covid_cases[df_covid_cases.location=="Nepal"]
# df_covid_cases[df_covid_cases.location=="Nepal"].groupby('date').total_cases.sum().astype(int)
nepal_cases = df_covid_cases[df_covid_cases.location=="Nepal"].groupby('date').total_cases.sum().astype(int)
nepal_cases.index = pd.to_datetime(nepal_cases.index) 
nepal_cases

date
2020-01-01        0
2020-01-10        0
2020-01-11        0
2020-01-12        0
2020-01-13        0
              ...  
2020-08-07    21750
2020-08-08    22214
2020-08-09    22592
2020-09-01    39460
2020-09-02    40529
Name: total_cases, Length: 247, dtype: int32

In [33]:
writer = pd.ExcelWriter(new_excel_file, engine="xlsxwriter")
sheet_name = "nepal_cases"
nepal_cases.to_excel(writer,sheet_name = sheet_name)

# Get the workbook
workbook = writer.book 
#get the sheet name
worksheet = writer.sheets[sheet_name]

# Create a chart object.
chart = workbook.add_chart({'type':'line'})

# configure the series of the chart from the dataframe data
chart.add_series({
    'values':"='"+sheet_name+"'!$B$2:$B$248",
    'categories':"='"+sheet_name+"'!$A$2:$A$248"
})

## add axis labels
chart.set_x_axis({
        'name':'Date',
        'num_font':{'rotation':45}
})
chart.set_y_axis({'name':'Total Cases','major_gridlines':{'visible':False}})

#insert the chart into the worksheet
worksheet.insert_chart('D2', chart)

writer.save()

## Multiple-Line Chart

```chart = workbook.add_chart({'type':'line'})```  


![Multiple line chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_legend.png)

In [43]:
#get world data
df_covid_cases.date = pd.to_datetime(df_covid_cases.date)
df_total_cases = df_covid_cases[['date','location','total_cases']].fillna(0)
world_cases =  df_total_cases.pivot(index='date',columns='location',values='total_cases').fillna(0).astype(int)
# world_cases.index = pd.to_datetime(world_cases.index)  
world_cases = world_cases[world_cases.index>='2020-8-01']
world_cases

location,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,...,Uruguay,Uzbekistan,Vatican,Venezuela,Vietnam,Western Sahara,World,Yemen,Zambia,Zimbabwe
date,,,,,,,,,,,,,,,,,,,,,
2020-08-01,36710,5276,30394,925,1109,3,91,185360,38841,121,...,1264,24304,12,18574,558,766,17577353,1728,5963,3169
2020-08-02,36710,5396,30950,925,1114,3,91,191289,39050,121,...,1278,25040,12,19443,590,766,17844982,1730,6228,3659
2020-08-03,36710,5519,31465,925,1148,3,91,196530,39102,122,...,1286,25828,12,20206,621,766,18068725,1734,6347,3921
2020-08-04,36747,5620,31972,937,1164,3,92,201906,39102,124,...,1291,26550,12,20206,652,766,18273389,1738,6580,4075
2020-08-05,36829,5750,32504,939,1164,3,92,206730,39298,132,...,1300,27314,12,21438,672,766,18529064,1760,6580,4221
2020-08-06,36896,5889,33055,939,1344,3,92,213522,39586,171,...,1309,28069,12,22299,717,766,18806477,1764,7022,4339
2020-08-07,36937,6016,33626,944,1395,3,92,220669,39819,263,...,1318,28809,12,23280,750,766,19092640,1767,7164,4395
2020-08-08,37015,6151,34155,955,1483,3,92,228182,39885,396,...,1325,29459,12,24166,789,766,19368549,1796,7486,4451
2020-08-09,37054,6275,34693,956,1483,3,92,235664,40185,509,...,1335,30197,12,24166,812,766,19637059,1800,7903,4575


In [45]:
writer = pd.ExcelWriter(new_excel_file, engine="xlsxwriter")
sheet_name = "world_cases"
world_cases.to_excel(writer,sheet_name = sheet_name)


# Get the workbook
workbook = writer.book 

#chartsheet 
chartsheet = workbook.add_chartsheet('world_cases_chart')

#get the sheet name
worksheet = writer.sheets[sheet_name]

# Create a chart object.
chart = workbook.add_chart({'type':'line'})

# configure the series of the chart from the dataframe data
for i in range(1, len(world_cases.columns.values)):
    chart.add_series({
        'name'      : [sheet_name, 0, i],
        'values'    : [sheet_name, 1, i, 248, i],
        'categories': [sheet_name, 1, 0, 248, 0]
    })

## add axis labels
chart.set_x_axis({
        'name':'Date',
        'num_font':{'rotation':45}
})
chart.set_y_axis({'name':'Total Cases','major_gridlines':{'visible':False}})

# #insert the chart into the worksheet
chartsheet.set_chart(chart)

writer.save()

## Scatter Chart

![Scatter chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_scatter.png)

In [220]:
#get world data
df_covid_cases.date = pd.to_datetime(df_covid_cases.date)
df_total_cases = df_covid_cases[['date','location','total_cases']][df_covid_cases.date==df_covid_cases.date.max()].fillna(0)
# world_cases =  df_total_cases.pivot(index='date',columns='location',values='total_cases').fillna(0).astype(int)
latest_data = df_total_cases.groupby('location').total_cases.sum()
latest_data

location
Afghanistan          38205.0
Albania               9606.0
Algeria              44833.0
Andorra               1184.0
Angola                2654.0
                     ...    
Western Sahara         766.0
World             25776601.0
Yemen                 1962.0
Zambia               12381.0
Zimbabwe              6559.0
Name: total_cases, Length: 210, dtype: float64

In [224]:
writer = pd.ExcelWriter(new_excel_file, engine="xlsxwriter")
sheet_name = "latest_cases"

#SET data in excel
latest_data.to_excel(writer,sheet_name = sheet_name)


# Get the workbook
workbook = writer.book 

#chartsheet 
# chartsheet = workbook.add_chartsheet('latest_cases')

#get the sheet name
worksheet = writer.sheets[sheet_name]

# Create a chart object.
chart = workbook.add_chart({'type':'scatter'})

# configure the series of the chart from the dataframe data
chart.add_series({
#     'name'      : [sheet_name, 1, 1, 210, 1],
    'values'    : [sheet_name, 1, 1, 211, 1],
    'categories': [sheet_name, 1, 0, 211, 0],
    'marker'    : {'type':'circle', 'size':7}
})

## add axis labels
chart.set_x_axis({
        'name':'Countries',
        'num_font':{'rotation':45}
})
chart.set_y_axis({'name':'Total Cases','major_gridlines':{'visible':False}})

# #insert the chart into the worksheet
worksheet.insert_chart('D2', chart)

writer.save()

## Other Charts that can be created
Referenced from : https://pandas-xlsxwriter-charts.readthedocs.io/chart_examples.html

## Colours 
![Colours](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_colors.png)  

## Area Chart
![Area Chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_area.png)   

## Stacked Area Chart
![Stacked Area Chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_stacked_area1.png)  

## Heat Stacked Area Chart
![Heat Stacked Area Chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_stacked_area2.png)  

## Stacked Column Chart
![Stacked Column Chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_stacked_column_farms.png)  

## Grouped Column Chart
![Grouped Column Chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_grouped_column.png)  

## Pie Chart
![Pie Chart](https://pandas-xlsxwriter-charts.readthedocs.io/_images/chart_pie.png)  